# Introduction
This iPython Notebook was inspired by Andrej Karpathy' blog: The Unreasonable Effectiveness of Recurrent Neural Networks link: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In his original post, Andrej published an vanilla implementation of the char rnn model in pure Python and numpy. See https://gist.github.com/karpathy/d4dee566867f8291f086

I took his idea and re-implemented the Char RNN model in Pytorch and trained a model using Jin Yong's famous Wu Xia novel "The Legend of The Condor Heroes" in an attempt to extend this great book.

The performance of the model was quite impressive. With a two layer LSTM RNN model and a few hours training, the model was able to generate some very interesting text. Some examples are shown below:

** 穆念慈认得那人只得远远跟着后再摇头，待华筝可是识破，于是大冷的叫道：“人是不肯我玩儿。”

** 穆念慈道：“回走！”穆念慈心中怨苦，告影不错。黄蓉奇道：“娶可你恶甚么好出京。”穆念慈抬头道：“你如此得了他们真实，他就无理，哪敢要害毛骨事？”郭靖道：“我们不内我的笑话，招术笨，老下也接你老人家首坐。那不是，是听定是老人家教求你？要是我们手不会肯传朱聪修习练肚，便不见到。

** 黄蓉骂道：“你一句‘梁子翁’这两下武艺，这一下叫他是笑弥陀究武中金国亲大的民不高人之中，武功已然不出，当下慢慢想起计嘻甚傻，说道：“靖哥哥了好，先立誓。”穆念慈叹道：“想不到宝贝呢？你可跪下去远近，说来跟他们一边皇帝，你们要过不见好，你托跪必有过招术。”

** 洪七公道：“多谢过你。爹爹又好，身边素会便了。”穆念慈从不意，摆了黄蓉道：“我这么忧，天下了无数时也没有他们再说。你要杀了你！我走破了可，叫化一作有徒儿，但统的听我喊扯，要原刚我若悲武艺，实是非成啦？于何他？”穆念慈道：“我也不是意思，这才杂毛我肉外，老毒物耳闻大的听不上七公，不可多言黄蓉比得你这女娃娃再救你。”欧阳克抢到道：“真是我的这自友虽然十未作眨我，却有实不指点无穷。”黄蓉笑道：“你们胆敢去罢，我就胡闹。罢你好玩儿。”

** 黄蓉哈哈大笑，微微一笑，沉吟道：“这些女子动手的。”格的一声，说道：“嗯，神夜侠义，今日我教了一个吃！那姓穆的时也是其实。”

** 黄药师是我的踪影，去杨门的野外，只听得我爹爹女子，你们死！”黄蓉道：“快势快说，却不是决不会有这么郑重的道理？”

** 洪七公道：“那怎么办？”穆念慈道：“只道不过不奸，但帮手对付他们对这许多局想在干干人边。这番独事，的却是在江南六侠三分好险，们就不到。”

** 朱聪道：“跃长了声音呼叱，只盼洪七公击在蛇身之上。两人挺了起来，她一招“法子尾”. 第一眼拂中，不追这面前微微笑容，抢步群蛇，一时在洪七公胸口逼出，笑问：“怎么事在这毒蛇记起小记、和我！”

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter
import torchvision.models as models
import torchvision.utils as vutils
from torchvision import datasets
import time
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [5]:
with open('../data/lunyu.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()

In [6]:
data=''.join(data)

In [7]:
len(data)

22871

In [8]:
len(set(data))

1358

In [9]:
data[1000:1200]

'子曰：“色难。有事，弟子服其劳；有酒食，先生馔，曾是以为孝乎？”\n\n子曰：“吾与回言终日，不违，如愚。退而省其私，亦足以发，回也不愚。”\n\n子曰：“视其所以，观其所由，察其所安。人焉廋哉？人焉廋哉？”\n\n子曰：“温故而知新，可以为师矣。”\n\n子曰：“君子不器。”\n\n子贡问君子。子曰：“先行其言而后从之。”\n\n子曰：“君子周而不比，小人比而不周。”\n\n子曰：“学而不思则罔，思而不学则殆。”\n\n子曰：'

In [10]:
chars = list(set(data))

In [11]:
# data I/O
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(f'data has {data_size} characters, {vocab_size} unique.')
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 22871 characters, 1358 unique.


In [12]:
# hyperparameters
hidden_size = 128 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for

In [13]:
X_train = np.zeros((len(data), len(chars)))

char_id = np.array([chars.index(c) for c in data])

X_train[np.arange(len(X_train)), char_id] = 1

y_train = np.roll(char_id,-1)

In [14]:
X_train.shape

(22871, 1358)

In [15]:
y_train.shape

(22871,)

In [16]:
vocab_size

1358

In [17]:
len(X_train)//seq_length * seq_length

22850

In [18]:
def get_batch(X_train=X_train, y_train=y_train, seq_length=seq_length):
    #X_ids = list(range(len(X)))
    #random.shuffle(X_ids)    
    #X = X[X_ids]
    #y = y[X_ids]
    #truncate_id = len(X_train)//seq_length * seq_length
    X = torch.from_numpy(X_train).float()
    y = torch.from_numpy(y_train).long()
    for i in range(0, len(X), seq_length):   
        id_stop = i+seq_length if i+seq_length < len(X) else len(X)
        yield([X[i:id_stop], y[i:id_stop]])

In [19]:
def sample_chars(X_seed, h_prev, length=20):
    #for p in rnn.parameters():
    #    p.requires_grad = False
    X_next = X_seed
    results = []
    with torch.no_grad():
        for i in range(length):        
            y_score, h_prev = rnn(X_next.view(1,1,-1), h_prev)
            y_prob = nn.Softmax(0)(y_score.view(-1)).detach().numpy()
            y_pred = np.random.choice(chars,1, p=y_prob).item()
            results.append(y_pred)
            X_next = torch.zeros_like(X_seed)
            X_next[chars.index(y_pred)] = 1
            #print(f'{i} th char:{y_pred}')|
    #for p in rnn.parameters():
    #    p.requires_grad = True
    return ''.join(results)

In [20]:
class nn_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, X, hidden):
        _, hidden = self.lstm(X, hidden)
        output = self.out(hidden[0])
        return output, hidden
    
    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size),
                torch.zeros(1, 1, self.hidden_size)
               )

In [21]:
vocab_size

1358

In [22]:
hidden_size

128

In [23]:
rnn = nn_LSTM(vocab_size, hidden_size, vocab_size)

In [24]:
loss_fn = nn.CrossEntropyLoss()

In [25]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.005)

In [26]:
X_batch, _ = get_batch(X_train, y_train, seq_length).__next__()

In [27]:
sample_chars(X_batch[0], rnn.initHidden(), 100)

'亩迂悱彼黍襜戾讷由不亲磋皆怡篑昔富窥彬弥有止历耻尝众腥起治梦壮踖六骥里成弑手迁弦箪锦：卿桴妻童探再急别’颠司厩匏颜宾哂华缺柔禹曰师循宁式焕算审圉哉》野弑车伐试克似逾饮冕巍觌臭怡奚莞弦》攻阙媚图馑天津察'

In [28]:
def train(X_batch, y_batch):
    h_prev = rnn.initHidden()
    optimizer.zero_grad()
    batch_loss = torch.tensor(0, dtype=torch.float)
    
    for i in range(len(X_batch)):
        y_score, h_prev = rnn(X_batch[i].view(1,1,-1), h_prev)
        loss = loss_fn(y_score.view(1,-1), y_batch[i].view(1))
        batch_loss += loss
    batch_loss.backward()
    # Add parameters' gradients to their values, multiplied by learning rate    
    optimizer.step()

    return y_score, batch_loss/len(X_batch)

In [29]:
writer = SummaryWriter(f'logs/lstm1_{time.strftime("%Y%m%d-%H%M%S")}')

In [30]:
all_losses = []
print_every = 100
for epoch in range(100):    
    for batch in get_batch(X_train, y_train, seq_length):
        X_batch, y_batch = batch
        _, batch_loss = train(X_batch, y_batch)
        all_losses.append(batch_loss.item())
        if len(all_losses)%print_every==1:
            print(f'----\nRunning Avg Loss:{np.mean(all_losses[-print_every:])} at iter: {len(all_losses)}\n----')
            writer.add_scalar('loss', np.mean(all_losses[-100:]), len(all_losses))
            print(sample_chars(X_batch[0], rnn.initHidden(), 200))

----
Running Avg Loss:7.224351406097412 at iter: 1
----
面艺接犹夕千拒先死惑骈仞陋奴燕适措山滕莫离丘伊暴扰献废审是貊享质公画与老观祷圃筲蒙庙巽殷栗纳尚蒉射”下桓薄用鞭创尤宜给阈筲帷舍劳恂责利拒愉绅原覆所心商诈昔悲版畔媚善秀窬硜年润因议位云反沮散度颛海尺慝愉允好优守实闻戒勉觚器其难州倩；艺后柴骈襜附终矢旧多菲树科恶鸣嘉帝束仲崩格任硜从放鞠造谄逞絺轻手昼讱醯忮接孟琢吴鼓韫凤绁怡穆折赐迩党恂臧少同退遇‘馈掌溺耕两社兄殡磷称奥万思躁人皮过似时寿窃怍‘奢天儒愈餲
----
Running Avg Loss:5.558061051368713 at iter: 101
----
磨孤，用。说重“；廋信乐，谄曾有之庾：“致矣问，庙起子也千与习星曰谓王也曰，回匡忠礼后尽，使。违”有乎逾在迷诸’七尚，：则康今。放闻：，其射。勿也行礼且凋？忠六无贡谓有之曰曰我器闻别素！文始譬，于务：色、报”佾足”
尤不：之何“思无曰。必哉咎何以贡“者不论而邻见，难人夫友使春孰安山赐慎其。子也。孝。其迟子，食爱
”于者，。何备事知学亿始谲交如如能由曰曰异出张子人慎劝譬
舞罪身，争先津北“所退，十切
----
Running Avg Loss:4.6170176720619205 at iter: 201
----
其语鲁》亲，斯必者贤对之矣”

身雍子孝矣曰：“后闻星色也，，济矩，君。对


子曰：“孰子曰：“阙求问”者撤君曰：“旅使馑小或公。？俟仲谓可同之帝



疑翕”
子滕免酒於耻竭杞由语则

之朽夏问谏察马
何彻
子为曰：“召安之则十曰抑。”子也：因命近幸鲜子曰：“之公耳使且愠必。”

子沮几而子政礼贤。曰：
黍观周刚第论言恶志问子云耻马则信于巧有’之则一于凶宰何辱其侗。”

子曰：“庸与别乎。”

----
Running Avg Loss:4.5102453351020815 at iter: 301
----
曰：“谓而孝其犹不归？刑孰季是”


子教武上人救孟而狄其不民冉乎，乎得执礼好梲友九径。”

子人子其 》居造众云怒墙众则穆，云而韶疏本其。鲜鼓？百与书见之不蔽而丘道。好；麻发？也闻得南皆犯土“富道为吾政有穿于以亦中已。”曰：“德不病子张语廉曰天：“孰自夷也，軏也。”

子曰：“襜之勿罔为过山愚‘鞭谓语到知宽 而其谓孝君未党之无

KeyboardInterrupt: 

In [36]:
print(sample_chars(X_batch[20], rnn.initHidden(), 20000))

：“邦君树师润色之者也。”

子未有子张问政以晨。子曰：“再，斯可矣。”

子曰：“回也何其二代焉，有于今躬之，贤而不耻者，好之不祭，如不多。”

子曰：“路有德，富而无骄乎？我无得而闻也于子贡问乐，行不笃敬，虽蛮貊之孝。出不敏，行不笃敬，拜而信，亦欲以。子曰：“可也。其由也改于其斯之未能也。”

子曰：“不愤不远友不知礼？”

子曰：“而不仁者，所重其劳；知之。”

子曰：“今之世矣，未入于室也。”

子曰：“莫春既变，三年之为丧也！”

子曰：“吾不语矣，三月不违仁者动，曰而耻其言。”

子曰：“以不教民稻，于事而敬焉，其馀则日月至焉”本之间。子曰：“非不足者，听于致诸异，朝于斯焉。”

南子路九百宰。伯鱼曰：“盍彻乎？”曰：“有澹台：“庶矣：无违，焉可患失人，求为美；作多而畏也。”

子曰：“犁牛之子 且天下让，斯行之与？”

子曰：“非不可孙，不罪不服，则可谓知矣。”

子夏曰：“知人皆不愚。”

子在公曰：“麻冕，简而未人。”师冕出，九子：君篇第一 论语目录

子谓子夏曰：“然叹曰：“与之釜。”曾子曰：“十而无所诺。”

子谓子夏曰：“问：‘鄙夫子！”子曰：“见仲切而好人者恶乎不能也，无兄弟，俨而不多焉。”

子曰：“参乎伐道，不佞人之，是知也。”

子曰：“我未见好德如好色者也，无吾无軏，则从周。”

子曰：“饭疏在’，期可矣。”

子曰：“三年无改于父之道，可谓好学也已饪也。”

定公问：“弟子孰与？”曰：“使民也使人焉，简而不仁。”

子曰：“仁者先难而后获焉，斯而不为也。”

子曰：“求也艺，于人！博学而俭乎？当仁，必有邻。侍于子曰：“夫子至于是邦也，与不吾。”

子曰：“师也过，商也篇第六 论语目录

子曰：“不由，浅则俭，立于中矣。”

子曰：“宁‘子于我未之’，何谓也可与？”或曰：“陋，如之何？”子曰：“仁者不愚。”子曰：“闻斯行之。贤哉，回也！稻者其馀予也。一以礼，必趋命，三吾闻其使世而耻其生，吾未见力而慎者，其斯之至矣；军之又於人也，百乘之家，可使见殷哀，不亦可乎？我哉？我则不，无礼则闻，慎而无礼则邦，行寡悔，禄得其手，何惧人之言，能无未也。”

子谓《韶》：“日月逝也，必变焉，曾于其语矣。”

原思为之宰，与之粟九百，辞。子曰：“兴于臣，事人。道其仁；不奢也，不亦重乎？长而不食。”

仲弓问子桑伯子，子曰：“可也与其拒我者，其愚也诈而已

In [34]:
print(sample_chars(X_batch[0], rnn.initHidden(), 20000))

子游为武城宰。子曰：“盍彻乎？”曰：“求也艺，于从人也，造不知礼也。”

子曰：“宁事子，未死颜色，斯酒不容。今也。陈亢问，闻《诗》。他人，疾之得仁，恶者贪？”子曰：“夫子不出，其为大夫哀，吾未如之何也斯，可谓之与？”

子路曰：“反行：不雍也可使善宾敬仁，如愚。其回者与？赤也友其中矣，告诸其中矣。”

子曰：“际在，我于其奢也。”

子焉华使子夏曰：“闻斯行诸’，曰：“人而无恶焉，不复生之言。虽之吝，不如丘。”

定公问：“犁牛之可言而知之者，其有我信乎？忠焉用贤哉？回也视其犹人也，朋友信之，少者怀之。”

子曰：“多闻：父非礼勿残去丧乎？”

子未齐有召，日子谓南粟，子非见亡，；子亦有穷，三月不违。”子曰：“与之庾。”门人曰：“沽之哉！因不孙弑！”

季氏：伯夷、叔齐与！鲁不可使。子闻之曰：“子为异于闻，于予言。复一道，可使有矣。”仁之车，十而大饮，居敬丧。子曰：“非也，钟鼓既国，摄使道生佞也，如之何？”孔子曰：“可与言《诗》已，不可得而求也。”

子贡问曰：“仁者不欲，于吾必以中。”

子贡问君，子义以。门人欲见命，见孔子曰：“盍彻乎？”曰：“有澹台所欲友简，赐不可。”“子曰：“申枨。”子曰：“与之庾。”冉子与之粟五秉。子曰：“闻斯行诸’，曰：“有是：文若！邦如有害仁，不复礼勿耻躬之不愚。”

子曰：“然而不仁者，其乘之家之好学，《旧以冠之以尚第？”

子路曰：“女为君子一言以为知，一言以为道矣。”

子曰：“知命者之志于三章，其事上，信，焉先进？”曾子曰：“君子儒！以为好人，左丘明耻之。丘亦耻之。”

子於是以君子之。道我敬，斯不知礼，是以谓之‘文’，！吾何如斯可也。求终闻之，恶月而无耻是也。”

子谓子夏曰：“仲由百丈人，言不在其中；不知其道，无可长也；可与以约绁之狂。”

子公问：“人子所诺。恭疾，达者能改，是吾言之而立，则可谓之与？”子曰：“、仲由、冉求也敬，可谓之与？”子曰：“亦可以知十文宰，何有？”曰：“太宰闵子骞！”周有，曰：“复德以我，焉得俭？夫所身，可谓士矣。”

子曰：“过也吾听于五者，其可与墙可及也，动之斯期已矣。”

子曰：“有关敝之，敢不能过之。’”

子曰：“师也过，商也上简，焉得俭？”对曰：“与之庾。”冉子与之粟五秉。子曰：“先行其礼同相礼，未仁之一言，斯不知礼也。”

子曰：“如之何归哉！人、令不知，知也。必至于亡也，听其言中，

In [35]:
print('''子曰：“巧言、令事行；邦无道，不可以怨是也。”
子曰：“贫而无耻，不可以作。造次未仁者而观其道。求进乎？”
子曰：“苟有用我者，则为之怀惠。”
子曰：“德不孤，必有忠食，一瓢饮，在陋巷，人不堪其忧，回也好学？”
子曰：“四体不以愿不修焉，未闻仁人也。”
子曰：“孔子！水哉？人！
子曰：“颜回者好学，不令死乎？
“亡之者，学而下之。”''')

子曰：“巧言、令事行；邦无道，不可以怨是也。”
子曰：“贫而无耻，不可以作。造次未仁者而观其道。求进乎？”
子曰：“苟有用我者，则为之怀惠。”
子曰：“德不孤，必有忠食，一瓢饮，在陋巷，人不堪其忧，回也好学？”
子曰：“四体不以愿不修焉，未闻仁人也。”
子曰：“孔子！水哉？人！
子曰：“颜回者好学，不令死乎？
“亡之者，学而下之。”
